In [4]:
!pip install transformers torch -q > /dev/null 2>&1

import re
from collections import defaultdict
from datetime import datetime

RED_FLAG_PATTERNS = {
    "danger_words": {
        "immediate": ["suicidal", "homicidal", "dying", "kill", "bleeding", "stroke",
                      "heartattack", "seizure", "unconscious", "911", "ambulance"],
        "severe_pain": ["excruciating", "agonizing", "unbearable", "torturous", "worst"],
        "respiratory": ["suffocating", "drowning", "gasping", "choking", "asphyxiating"],
        "neuro": ["paralyzed", "paralysis", "numbness", "visionloss", "facedrooping"],
        "trauma": ["amputated", "crushed", "fractured", "hemorrhage", "bleedingout"]
    },

    "danger_phrases": {
        "cardiac": ["chest pain radiating", "pressure in chest", "feeling of doom",
                   "heart skipping beats", "left arm pain", "jaw pain chest"],
        "neuro": ["worst headache ever", "thunderclap headache", "sudden vision loss",
                 "face drooping", "arm weakness", "speech difficulty"],
        "respiratory": ["can't breathe", "struggling for air", "lying down breathless",
                       "turning blue", "lips blue", "choking sensation"],
        "abdominal": ["rigid abdomen", "vomiting blood", "black tarry stool",
                     "unable to pass gas", "severe abdominal pain"],
        "psychiatric": ["want to die", "end my life", "kill myself", "harm myself",
                       "thoughts of suicide", "voices telling me"]
    },

    "sentence_patterns": [
        r"my chest feels like.*(elephant|weight|crushing)",
        r"I have pain.*(radiating|going to).*(arm|jaw|back)",
        r"this is.*worst.*headache.*(life|ever)",
        r"I can't.*(move|feel).*(arm|leg|face)",
        r"I feel like.*(drowning|suffocating|choking)",
        r"my.*(lips|skin).*(blue|purple)",
        r"I want to.*(die|end it|kill myself)",
        r"there is.*(lot of|uncontrolled).*bleeding"
    ]
}

class SimpleRedFlagDetector:
    def __init__(self):
        self.patterns = RED_FLAG_PATTERNS
        self.sentence_regex = [re.compile(p, re.IGNORECASE) for p in self.patterns["sentence_patterns"]]
        self.danger_word_set = set()
        for words in self.patterns["danger_words"].values():
            self.danger_word_set.update(words)

    def detect(self, text: str):
        text_lower = text.lower()

        word_results = self._detect_words(text_lower)
        phrase_results = self._detect_phrases(text_lower)
        sentence_results = self._detect_sentences(text_lower)

        risk_level = self._calculate_risk(word_results, phrase_results, sentence_results)

        return {
            "text": text,
            "risk_level": risk_level,
            "word_matches": word_results["found"],
            "phrase_matches": phrase_results["found"],
            "sentence_matches": sentence_results["found"],
            "escalation_reasons": self._get_reasons(word_results, phrase_results, sentence_results)
        }

    def _detect_words(self, text: str):
        words = text.split()
        found = []
        categories = defaultdict(list)

        for word in words:
            if word in self.danger_word_set:
                found.append(word)
                for cat, word_list in self.patterns["danger_words"].items():
                    if word in word_list:
                        categories[cat].append(word)
            else:
                for danger_word in self.danger_word_set:
                    if danger_word in word:
                        found.append(word)
                        for cat, word_list in self.patterns["danger_words"].items():
                            if danger_word in word_list:
                                categories[cat].append(word)

        return {"found": list(set(found)), "categories": dict(categories)}

    def _detect_phrases(self, text: str):
        found = []
        categories = defaultdict(list)

        for category, phrases in self.patterns["danger_phrases"].items():
            for phrase in phrases:
                if self._contains_phrase(phrase, text):
                    found.append(phrase)
                    categories[category].append(phrase)

        return {"found": found, "categories": dict(categories)}

    def _detect_sentences(self, text: str):
        found = []
        for i, pattern in enumerate(self.sentence_regex):
            match = pattern.search(text)
            if match:
                found.append({
                    "pattern_id": i,
                    "matched": match.group(0)[:100],
                    "full_pattern": self.patterns["sentence_patterns"][i]
                })
        return {"found": found}

    def _contains_phrase(self, phrase: str, text: str):
        phrase_words = phrase.split()
        return all(word in text for word in phrase_words)

    def _calculate_risk(self, word_res, phrase_res, sentence_res):
        score = 0
        if word_res["found"]:
            score += 3
        if phrase_res["found"]:
            score += 2
        if sentence_res["found"]:
            score += 4

        if score >= 6:
            return "CRITICAL"
        elif score >= 3:
            return "HIGH"
        elif score >= 1:
            return "MEDIUM"
        return "LOW"

    def _get_reasons(self, word_res, phrase_res, sentence_res):
        reasons = []
        if word_res["found"]:
            reasons.append(f"Danger words: {', '.join(word_res['found'][:3])}")
        if phrase_res["found"]:
            reasons.append(f"Red flag phrases: {', '.join(phrase_res['found'][:2])}")
        if sentence_res["found"]:
            for match in sentence_res["found"][:2]:
                reasons.append(f"Pattern: '{match['matched'][:50]}...'")
        return reasons

def integrate_with_symptom_checker(symptoms_text: str, original_risk: str = "LOW"):
    detector = SimpleRedFlagDetector()
    results = detector.detect(symptoms_text)

    red_flag_risk = results["risk_level"]
    risk_hierarchy = ["LOW", "MEDIUM", "HIGH", "CRITICAL"]

    original_idx = risk_hierarchy.index(original_risk.upper())
    red_flag_idx = risk_hierarchy.index(red_flag_risk.upper())
    final_idx = max(original_idx, red_flag_idx)
    final_risk = risk_hierarchy[final_idx]

    response = {
        "final_risk": final_risk,
        "was_escalated": red_flag_idx > original_idx,
        "red_flag_details": results,
        "timestamp": datetime.now().isoformat(),
        "actions": get_recommended_actions(final_risk, results)
    }

    if final_risk in ["HIGH", "CRITICAL"]:
        response["queue_priority"] = "URGENT"
        response["alert_doctor"] = True

    return response

def get_recommended_actions(risk: str, results):
    if risk == "CRITICAL":
        return [
            "CALL EMERGENCY SERVICES (911)",
            "Do not wait for doctor review",
            "Patient needs immediate medical attention"
        ]
    elif risk == "HIGH":
        return [
            "Urgent doctor review required",
            "Move to top of pending queue",
            "Consider emergency consultation"
        ]
    elif results["risk_level"] == "MEDIUM":
        return ["Review with priority"]
    return ["Normal review process"]

if __name__ == "__main__":
    print("RED FLAG DETECTION SYSTEM")
    print("--------------------------")

    sample_symptoms = "I have crushing chest pain radiating to my jaw, feels like an elephant on my chest"

    detector = SimpleRedFlagDetector()
    results = detector.detect(sample_symptoms)

    print(f"Input: {sample_symptoms}")
    print(f"Risk Level: {results['risk_level']}")
    print(f"Danger Words: {results['word_matches']}")
    print(f"Danger Phrases: {results['phrase_matches']}")

    integrated_result = integrate_with_symptom_checker(sample_symptoms, "MEDIUM")
    print(f"Original Risk: MEDIUM")
    print(f"Final Risk: {integrated_result['final_risk']}")
    print(f"Escalated: {integrated_result['was_escalated']}")
    print(f"Actions: {integrated_result['actions']}")

RED FLAG DETECTION SYSTEM
--------------------------
Input: I have crushing chest pain radiating to my jaw, feels like an elephant on my chest
Risk Level: MEDIUM
Danger Words: []
Danger Phrases: ['chest pain radiating', 'jaw pain chest']
Original Risk: MEDIUM
Final Risk: MEDIUM
Escalated: False
Actions: ['Review with priority']
